In [1]:
# %pip install pandas
# %pip install transformers==4.45.2 sentence-transformers==3.1.1
# %pip install datasets
# %pip install transformers[torch]
# %pip install ipywidgets

In [2]:
import pandas as pd

q = pd.read_csv("questions.csv", encoding="utf-16")["0"]
a = pd.read_csv("answers.csv", encoding="utf-16")["1"]
n = len(a)

In [3]:
from sentence_transformers import SentenceTransformer, losses
import torch

model = SentenceTransformer('sentence-transformers/LaBSE', cache_folder="./cache")
train_loss = losses.CachedMultipleNegativesRankingLoss(model)

c:\Users\RUShulpiAn\Desktop\ServiceBot\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
# from tqdm import tqdm
# import numpy as np

# # Включаем tqdm для визуализации
# def encode_with_progress(model, texts):
#     embeddings = []
#     # Используем tqdm для отображения прогресса
#     for text in tqdm(texts, desc="Encoding texts", ncols=100, dynamic_ncols=True):
#         embedding = model.encode(text, show_progress_bar=False)  # Подключаем внутреннюю визуализацию
#         embeddings.append(embedding)
#     return np.array(embeddings)


# # Энкодим вопросы и ответы с визуализацией прогресса
# q_embeddings = encode_with_progress(model, [str(q[i]) if q[i] is not None else "" for i in range(n)])
# a_embeddings = encode_with_progress(model, [str(a[i]) if a[i] is not None else "" for i in range(n)])

In [5]:
from sentence_transformers import InputExample
from torch.utils.data import DataLoader

questions = [str(q[i]) if q[i] is not None else "" for i in range(n)]
answers = [str(a[i]) if a[i] is not None else "" for i in range(n)]

train_examples = [InputExample(texts=[questions[i], answers[i]]) for i in range(n)]
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=64)

In [ ]:
# Обучение с использованием всех ядер процессора
torch.set_num_threads(torch.get_num_threads())
num_epochs = 10

# Добавление tqdm для визуализации прогресса
for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    model.fit(
        train_objectives=[(train_dataloader, train_loss)],
        epochs=1,
        warmup_steps=100,
        show_progress_bar=True
    )

# Сохранение обученной модели
model.save('./sentence_bert_service_desk_model')

Epoch 1/3


  0%|          | 0/460 [00:00<?, ?it/s]

{'train_runtime': 59209.775, 'train_samples_per_second': 0.497, 'train_steps_per_second': 0.008, 'train_loss': 1.7783228335173233, 'epoch': 1.0}
Epoch 2/3


  0%|          | 0/460 [00:00<?, ?it/s]

KeyboardInterrupt: 